In [ ]:
import os
import numpy as np
import pandas as pd
import quantities as pq
import neo
import neurotic
import axographio

In [ ]:
export_dir = '../../data/data-for-paper'
if not os.path.exists(export_dir):
    os.mkdir(export_dir)

In [ ]:
metadata_file = '../../data/metadata.yml'

In [ ]:
epoch_types_to_keep = [
    'Swallow (regular 5-cm nori strip)',
    'Swallow (tape nori)',
    'B38 activity',
    'I2 protraction activity',
    'B8 activity',
    'B3/6/9/10 activity',
    'B4/B5 activity',
    'Inward movement',
    'Force shoulder end',
    'Force rise start',
    'Force plateau start',
    'Force plateau end',
    'Force drop end',
]

In [ ]:
channel_names_by_animal = {
    'Animal 1': ['I2-L', 'RN-L', 'BN2-L', 'BN3-L',    'Force'],
    'Animal 2': ['I2',   'RN',   'BN2',   'BN3',      'Force'],
    'Animal 3': ['I2',   'RN',   'BN2',   'BN3-PROX', 'Force'],
    'Animal 4': ['I2',   'RN',   'BN2',   'BN3-DIST', 'Force'],
    'Animal 5': ['I2',   'RN',   'BN2',   'BN3-PROX', 'Force'],
}

In [ ]:
datasets = {
    'Animal 1 Unloaded 1': ('IN VIVO / JG07 / 2018-05-20 / 002', [1169, 1191]), # 4 swallows
    'Animal 1 Unloaded 2': ('IN VIVO / JG07 / 2018-05-20 / 002', [1495, 1518]), # 4 swallows
    'Animal 1 Unloaded 3': ('IN VIVO / JG07 / 2018-05-20 / 002', [1581, 1615]), # 5 swallows
    'Animal 1 Loaded 1':   ('IN VIVO / JG07 / 2018-05-20 / 002', [2716, 2755]), # 5 swallows
    
    'Animal 2 Unloaded 1': ('IN VIVO / JG08 / 2018-06-21 / 002', [ 256,  287]), # 4 swallows
    'Animal 2 Unloaded 2': ('IN VIVO / JG08 / 2018-06-21 / 002', [ 454,  481]), # 4 swallows
    'Animal 2 Loaded 1':   ('IN VIVO / JG08 / 2018-06-21 / 002', [ 141,  210]), # 7 swallows, some bucket and head movement
    'Animal 2 Loaded 2':   ('IN VIVO / JG08 / 2018-06-21 / 002', [ 660,  701]), # 5 swallows, large bucket movement
    'Animal 2 Loaded 3':   ('IN VIVO / JG08 / 2018-06-21 / 002', [1448, 1477]), # 3 swallows, some bucket movement
    
    'Animal 3 Unloaded 1': ('IN VIVO / JG11 / 2019-04-03 / 001', [1791, 1819]), # 5 swallows
    'Animal 3 Unloaded 2': ('IN VIVO / JG11 / 2019-04-03 / 004', [ 551,  568]), # 3 swallows
    'Animal 3 Loaded 1':   ('IN VIVO / JG11 / 2019-04-03 / 004', [1226, 1280]), # 5 swallows, inward food movements had very low amplitude
    
    'Animal 4 Unloaded 1': ('IN VIVO / JG12 / 2019-05-10 / 002', [ 147,  165]), # 3 swallows
    'Animal 4 Unloaded 2': ('IN VIVO / JG12 / 2019-05-10 / 002', [ 228,  245]), # 3 swallows
    'Animal 4 Unloaded 3': ('IN VIVO / JG12 / 2019-05-10 / 002', [ 277,  291]), # 3 swallows
    'Animal 4 Loaded 1':   ('IN VIVO / JG12 / 2019-05-10 / 002', [ 434,  465]), # 4 swallows
    'Animal 4 Loaded 2':   ('IN VIVO / JG12 / 2019-05-10 / 002', [2897, 2937]), # 5 swallows
    
    'Animal 4 Unloaded 2 Superset': ('IN VIVO / JG12 / 2019-05-10 / 002', [ 223.4,  261.4]), # Fig 2A
    'Animal 4 Loaded 2 Superset':   ('IN VIVO / JG12 / 2019-05-10 / 002', [2875.3, 3039.3]), # Fig 2B & 1
    
    'Animal 5 Unloaded 1': ('IN VIVO / JG14 / 2019-07-30 / 001', [1834, 1865]), # 4 swallows
    'Animal 5 Unloaded 2': ('IN VIVO / JG14 / 2019-07-30 / 001', [1909, 1943]), # 5 swallows
    'Animal 5 Unloaded 3': ('IN VIVO / JG14 / 2019-07-30 / 001', [2049, 2084]), # 5 swallows
    'Animal 5 Loaded 1':   ('IN VIVO / JG14 / 2019-07-29 / 004', [ 828,  870]), # 5 swallows
}

In [ ]:
metadata = neurotic.MetadataSelector(metadata_file)

for label, (data_set_name, time_window) in datasets.items():
    metadata.select(data_set_name)
    
    if 'annotations_file' in metadata and metadata['annotations_file'] is not None:
        # read the file
        csv_file = metadata.abs_path('annotations_file')
        df = pd.read_csv(csv_file)
        
        # filter rows by time
        df = df[(df['Start (s)'] >= time_window[0]) & (df['End (s)'] <= time_window[1])]
        
        # filter rows by epoch type
        df = df.query(f'Type in {epoch_types_to_keep}')
        
        # export the filtered annotations
        export_file = os.path.join(export_dir, f'{label} Annotations.csv')
        df.round(6).to_csv(export_file, index=False)

    if 'epoch_encoder_file' in metadata and metadata['epoch_encoder_file'] is not None:
        # read the file
        csv_file = metadata.abs_path('epoch_encoder_file')
        df = pd.read_csv(csv_file)
        
        # filter rows by time
        df = df[(df['Start (s)'] >= time_window[0]) & (df['End (s)'] <= time_window[1])]
        
        # filter rows by epoch type
        df = df.query(f'Type in {epoch_types_to_keep}')
        
        # export the filtered annotations
        export_file = os.path.join(export_dir, f'{label} Epoch Encoder.csv')
        df.round(6).to_csv(export_file, index=False)

    if 'data_file' in metadata and metadata['data_file'] is not None:
        # open the file
        axo_file = metadata.abs_path('data_file')
        r = neo.io.AxographIO(axo_file)
        blk = r.read_block(lazy=True, signal_group_mode='split-all')
        
        # load signal slices
        channels_to_keep = channel_names_by_animal[label[:8]]
        sigs = []
        for channel in channels_to_keep:
            sig = next((sig for sig in blk.segments[0].analogsignals if sig.name == channel), None)
            if sig is not None:
                t_start = time_window[0]*pq.s - 0.01*pq.ms
                t_stop = time_window[1]*pq.s + 0.01*pq.ms
                sigs.append(sig.time_slice(t_start, t_stop))
        
        # write the signal slices to an AxoGraph file
        export_file = os.path.join(export_dir, f'{label}.axgx')
        times = axographio.linearsequence(
            sigs[0].times.size,
            sigs[0].t_start.rescale('s').magnitude,
            sigs[0].sampling_period.rescale('s').magnitude
        )
        data = [times]
        names = ['Time (s)']
        for sig in sigs:
            index = np.where(r.header['signal_channels']['name'] == sig.name)[0][0]
            gain = r.header['signal_channels']['gain'][index]
            offset = r.header['signal_channels']['offset'][index]
            short_array = np.round((sig.as_array().flatten()-offset)/gain).astype(int)
            data.append(axographio.scaledarray(short_array, gain, offset))
            names.append(f'{sig.name} ({sig.dimensionality.string})')
        f = axographio.file_contents(names, data)
        f.write(export_file)

**IMPORTANT:** After writing the sliced AxoGraph files, each file should be opened in AxoGraph and re-saved to fix some missing metadata that is required for ``neo.rawio.AxographRawIO`` to read it. You may also want to execute the "Separate Traces" menu action so that the channels are not stacked.